# Scrape Online Travel Agent (OTA) Property Prices

@Nileka add a description here as to what this notebook offers

### Execute the cell below once to turn off debugging

In [2]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

## Logging Configuration
___TODO___ add logging of status

In [2]:
import logging


## Initialize the OTA scrape class

In [43]:
import sys
sys.path.insert(1, '../../lib/wrangler')
import otaWebScraper as otaws
import datetime

dataDirPath="../../data/hospitality/bookings/scraper"

if debug:
    import importlib
    otaws = importlib.reload(otaws)
    
clsScraper = otaws.OTAWebScraper()

All packages in OTAWebScraper loaded successfully!
Initialing OTAWebScraper class for  data


## Initialize the set of OTA URL for Scraping

In [17]:
from datetime import date

path = dataDirPath
file = "otaInputURLs.json"
start_date = date(2022,9,11)
end_date = date(2022,9,20)

try:
    if start_date < date.today():
        raise ValueError("Start date must be greater than today: %s" % str(date.today()))
    if end_date <= start_date:
        raise ValueError("End date %s is invalid. It must be greater than Start Date: %s" % (str(end_date),str(start_date)))
    scrape_crietia_dict = {"pageOffset":10,
                           "pageUpperLimit":550,
                           "startDate": start_date,
                           "endDate" : end_date,
             }
    _, _ota_url_parameterized_list  = clsScraper.build_scrape_url_list(dirPath=path,
                                                                       fileName=file,
                                                                       **scrape_crietia_dict)
except Exception as err:
    _s_fn_id = "Class <WebScraper> Function <read_folder_csv_to_df>"
    print("[Error]"+_s_fn_id, err)
    print(traceback.format_exc())

#print(_ota_url_parameterized_list)
print("Completed parameterizing urls with %d instances." % (len(_ota_url_parameterized_list)))

Loaded 1 properties to begin scraping OTA data.
Processing booking.com ...
Completed parameterizing urls with 7056 instances.


In [18]:
#path = dataDirPath
file = "otaInputURLs.json"
property_dict = clsScraper.load_ota_list(dirPath=dataDirPath, fileName=file)
_scrape_tags_df = clsScraper.get_scrape_html_tags(property_dict)
print(_scrape_tags_df)

        variable   tag                   code          ota
0  content_block   div            .d20f4628d0  booking.com
0  property_name  span  fcab3ed991 a23c043802  booking.com
0      room_type  span             df597226dd  booking.com
0      room_rate   div  fcab3ed991 bd73d13072  booking.com
0   review_score   div  b5cd09854e d10a6220b4  booking.com
0       location   div             a1fbd102d9  booking.com
0          Other   div             d22a7c133b  booking.com


## Prepare folder & file structure

In [19]:
from datetime import datetime, timezone

dirPath = None
_search_dt = datetime.now()
kwargs = {'search_datetime': _search_dt}
''' include the timezone '''
_search_dt = (_search_dt.replace(tzinfo=timezone.utc)).isoformat()

_current_search_data_dir = clsScraper.get_search_data_dir_path(dirPath, **kwargs)
print("Extracting data into %s for search datetime: %s" % (_current_search_data_dir,str(_search_dt)))

Extracting data into ../../data/hospitality/bookings/scraper/rates/2022-9-10-8-0/ for search datetime: 2022-09-10T08:05:15.179388+00:00


## Scrape data into CSVs
___TODO:___ hault if internet connection times out; might be possible with airflow.

In [22]:
_l_saved_files = clsScraper.scrape_url_list(_ota_url_parameterized_list, _search_dt, _current_search_data_dir)

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> HTTPSConnectionPool(host='www.booking.com', port=443): Max retries exceeded with url: /searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20079110&dest_type=city&checkin=2022-09-11&checkout=2022-09-12&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=0 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fa4997b3250>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
Traceback (most recent call last):
  File "/home/nuwan/.local/lib/python3.8/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/nuwan/.local/lib/python3.8/site-packages/urllib3/util/co

KeyboardInterrupt: 

## Read from CSV to Dataframe

In [44]:
#dirPath = _current_search_data_dir
dirPath = "../../data/hospitality/bookings/scraper/rates/2022-9-7-9-0/"
removed_files = clsScraper.remove_empty_files(dirPath)
if len(removed_files) > 0:
    print("%d files removed" %len(removed_files))
kwargs = {"columns" : ["search_datetime",
                       "checkin_date",
                       "propery_name",
                       "room_type",
                       "room_rate",
                       "review_score",
                       "destination_id",
                       "location",
                       "other"]}

rates_df = clsScraper.read_folder_csv_to_df(dirPath, **kwargs)

## Load into Database

In [4]:
import sys
sys.path.insert(1, '../../lib/wrangler')
import sparkwkloads as spark
import datetime

if debug:
    import importlib
    spark = importlib.reload(spark)

clsSparkWL = spark.SparkWorkLoads()
#_session = clsSparkWL.get_spark_session()

All packages in SparkWorkLoads loaded successfully!
22/09/10 23:18:06 WARN Utils: Your hostname, FarmRaiderTester resolves to a loopback address: 127.0.1.1; using 192.168.124.15 instead (on interface enp2s0)
22/09/10 23:18:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/09/10 23:18:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Connection complete! ready to load data.


In [22]:
dirPath = "../../data/hospitality/bookings/scraper/rates/2022-9-7-9-0/"
csv_data = clsSparkWL.read_csv_to_sdf(dirPath=dirPath)

In [23]:
from pyspark.sql.functions import substring,lit,col
from pyspark.sql.types import StringType,BooleanType,DateType,DecimalType,FloatType, IntegerType,LongType, ShortType, TimestampType

csv_data=csv_data.withColumn("Currency", lit("US$"))
csv_data=csv_data.withColumn('Room Rate', substring('Room Rate', 4,10))

csv_data = csv_data.withColumn("Currency",col("Currency").cast(StringType())) \
    .withColumn("Room Rate",col("Room Rate").cast(FloatType())) \
    .withColumn("jobStartDate",col("Search DateTime").cast(DateType()))

csv_data.printSchema()
csv_data.show(n=2, vertical=True, truncate=False)

root
 |-- Search DateTime: timestamp (nullable = true)
 |-- Checkin Date: timestamp (nullable = true)
 |-- Property Name: string (nullable = true)
 |-- Room Type: string (nullable = true)
 |-- Room Rate: float (nullable = true)
 |-- Review Score: double (nullable = true)
 |-- Destination ID: integer (nullable = true)
 |-- Location Desc: string (nullable = true)
 |-- Other Info: string (nullable = true)
 |-- Currency: string (nullable = false)
 |-- jobStartDate: date (nullable = true)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Search DateTime | 2022-09-07 17:24:08.173406                                                                                                                                                             
 Checkin Date    | 2022-09-08 00:00:00                                                                  

In [12]:
_s_tbl_name = "lakehouse.ota_scraped_property_prices"
data = clsSparkWL.get_data_from_table(dbTable=_s_tbl_name)
data.printSchema()

Wait a moment, retrieving data ...
Loading complete!
root
 |-- id: integer (nullable = true)
 |-- ota_name: string (nullable = true)
 |-- search_dt: array (nullable = true)
 |    |-- element: timestamp (containsNull = true)
 |-- property_name: string (nullable = true)
 |-- checkin_date: array (nullable = true)
 |    |-- element: timestamp (containsNull = true)
 |-- checkout_date: array (nullable = true)
 |    |-- element: timestamp (containsNull = true)
 |-- room_type: string (nullable = true)
 |-- room_rate: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- review_score: double (nullable = true)
 |-- destination_id: string (nullable = true)
 |-- location: string (nullable = true)
 |-- other: string (nullable = true)
 |-- datasource: string (nullable = true)
 |-- dataowner: string (nullable = true)
 |-- created_by: string (nullable = true)
 |-- create_dt: array (nullable = true)
 |    |-- element: timestamp (containsNull = true)
 |-- create_proc: string (nullable = 